# Import librarie

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# Define Model

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.1)              # Input  --> Output     | Rin --> Rout
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)  # 28x28x1 --> 28x28x8   | 1   --> 3 
        self.conv1_bn = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)  # 28x28x8 --> 28x28x8   | 3   --> 5
        self.conv2_bn = nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8, 8, 3, padding=1)  # 28x28x8 --> 28x28x8   | 5   --> 7
        self.conv3_bn = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2)             # 28x28x8 --> 14x14x8   | 7   --> 8
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1) # 14x14x8 --> 14x14x16  | 8   --> 12
        self.conv4_bn = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3)           # 14x14x16 --> 12x12x16 | 12  --> 16
        self.conv5_bn = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)             # 12x12x16 --> 6x6x16   | 16  --> 18 
        self.conv6 = nn.Conv2d(16, 32, 3)           # 6x6x16   --> 4x4x32   | 18  --> 26
        self.conv6_bn = nn.BatchNorm2d(32)
        self.conv7 = nn.Conv2d(32, 10, 3)           # 4x4x32   --> 2x2x10   | 26  --> 34

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = self.dropout(x)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = self.dropout(x)
        x = F.relu(self.conv3_bn(self.conv3(x)))
        x = self.pool1(x)
        x = self.dropout(x)
        x = F.relu(self.conv4_bn(self.conv4(x)))
        x = self.dropout(x)
        x = F.relu(self.conv5_bn(self.conv5(x)))
        x = self.pool2(x)
        x = self.dropout(x)
        x = F.relu(self.conv6_bn(self.conv6(x)))
        x = self.dropout(x)
        x = self.conv7(x)
        x = x.mean([2, 3])
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4            [-1, 8, 28, 28]             584
       BatchNorm2d-5            [-1, 8, 28, 28]              16
           Dropout-6            [-1, 8, 28, 28]               0
            Conv2d-7            [-1, 8, 28, 28]             584
       BatchNorm2d-8            [-1, 8, 28, 28]              16
         MaxPool2d-9            [-1, 8, 14, 14]               0
          Dropout-10            [-1, 8, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           1,168
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

<ipython-input-2-42104d8a943a>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1, verbose=True)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-42104d8a943a>:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
epoch=1 loss=0.1532626897096634 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 15.66it/s]



Test set: Average loss: 0.0724, Accuracy: 9769/10000 (97.69%)

Adjusting learning rate of group 0 to 1.0000e-02.


epoch=2 loss=0.05458409711718559 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.72it/s]



Test set: Average loss: 0.0704, Accuracy: 9800/10000 (98.00%)

Adjusting learning rate of group 0 to 1.0000e-02.


...


Test set: Average loss: 0.0193, Accuracy: 9943/10000 (99.43%)

Adjusting learning rate of group 0 to 1.0000e-03.


epoch=18 loss=0.06084350869059563 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.54it/s]



Test set: Average loss: 0.0188, Accuracy: 9945/10000 (99.45%)

Adjusting learning rate of group 0 to 1.0000e-03.


epoch=19 loss=0.011919829063117504 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.31it/s]



Test set: Average loss: 0.0193, Accuracy: 9946/10000 (99.46%)

Adjusting learning rate of group 0 to 1.0000e-03.
